# Scraping Amazon

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install amazoncaptcha

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd 
import numpy as np 
import requests
import sys
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from amazoncaptcha import AmazonCaptcha
import logging

In [4]:
logging.basicConfig(level=logging.INFO, filename="scraping_log.log",filemode="w",
                    format="%(asctime)s %(levelname)s %(message)s")

In [5]:
'''
with open('scraping_log.log', 'w'):
    pass # Clearing logs
'''

"\nwith open('scraping_log.log', 'w'):\n    pass # Clearing logs\n"

In [6]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск в фоновом режиме (без открытия браузера)

In [7]:
driver = webdriver.Chrome() # options=chrome_options

url = 'https://www.amazon.com/'

driver.get(url)
time.sleep(3)
logging.info("Amazon opened for getting categories")
# time.sleep(10) # Т.к. надо вручную пройти капчу (ну либо деньги платить за сервисы для ее автономного прохождения)

In [8]:
def SolveCaptcha(driver):
    try:
        logging.warning("Captcha solving started")
        link = driver.find_element(By.XPATH, "//div[@class = 'a-row a-text-center']//img").get_attribute('src')
    
        captcha = AmazonCaptcha.fromlink(link)
        
        captcha_value = AmazonCaptcha.solve(captcha)
        
        clickable = driver.find_element(By.ID, 'captchacharacters')
        ActionChains(driver) \
            .click(clickable) \
            .perform()
        
        time.sleep(1)
        
        ActionChains(driver)\
                .send_keys(captcha_value)\
                .perform()
        
        clickable = driver.find_element(By.XPATH, "//button[@class = 'a-button-text']")
        ActionChains(driver) \
            .click(clickable) \
            .perform()
        logging.info("Captcha solved")
    except Exception as e:
        logging.error(f"Error appeared while solving captcha", exc_info=e)
    return driver

In [9]:
driver = SolveCaptcha(driver)

In [10]:
def SetCountry(driver):
    logging.info("Setting country started")
    clickable = driver.find_element(By.ID, 'nav-global-location-popover-link')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    clickable = driver.find_element(By.ID, 'GLUXZipUpdateInput')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    ActionChains(driver)\
            .send_keys("10001")\
            .perform()
    
    time.sleep(1)
    
    clickable = driver.find_element(By.ID, 'GLUXZipUpdate')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    
    time.sleep(2)
    
    logging.info("Setting country ended")
    return driver

In [11]:
driver = SetCountry(driver)

In [12]:
clickable = driver.find_element(By.ID, 'nav-hamburger-menu')
ActionChains(driver) \
    .click(clickable) \
    .perform()

time.sleep(2)

In [13]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

df = pd.DataFrame({'Category': [], 'Subcategory': [], 'Link': []})

logging.info("Getting categories started")
for i in soup.find_all("ul", {'class': "hmenu hmenu-translateX-right"}):
    if int(i['data-menu-id']) > 11 and int(i['data-menu-id']) < 29:
        categ = i.find_all('div')[1].text
        if 'Whole Foods' in categ or 'Amazon' in categ or 'More to Explore' in categ:
            continue
        flag = False
        for j in i.find_all('li'):
            try:
                if int(j.find('div')['aria-level']) == 6:
                    if flag:
                        break
                    flag = True
            except:
                pass
            try:
                k = j.find('a')
                if k.text != 'main menu' and 'Amazon' not in k.text:
                    df.loc[len(df)] = [categ, k.text, url[0:-1] + k['href']]
            except:
                pass

logging.info("Getting categories ended")

In [14]:
df.head(20)

,Category,Subcategory,Link
0,"Clothing, Shoes, Jewelry & Watches",Women,https://www.amazon.com/gp/browse.html?node=714...
1,"Clothing, Shoes, Jewelry & Watches",Men,https://www.amazon.com/gp/browse.html?node=714...
2,"Clothing, Shoes, Jewelry & Watches",Girls,https://www.amazon.com/gp/browse.html?node=714...
3,"Clothing, Shoes, Jewelry & Watches",Boys,https://www.amazon.com/gp/browse.html?node=714...
4,"Clothing, Shoes, Jewelry & Watches",Baby,https://www.amazon.com/gp/browse.html?node=714...
5,"Clothing, Shoes, Jewelry & Watches",Luggage,https://www.amazon.com/gp/browse.html?node=947...
6,"Clothing, Shoes, Jewelry & Watches",Luxury Stores,https://www.amazon.com/luxurystores?ref_=nav_e...
7,"Clothing, Shoes, Jewelry & Watches",The Drop,https://www.amazon.comhttps://www.amazon.com/t...
8,"Clothing, Shoes, Jewelry & Watches",Sales & Deals,https://www.amazon.com/gp/browse.html?node=953...
9,Books,Books,https://www.amazon.com/gp/browse.html?node=283...


In [15]:
df.shape

(138, 3)

In [16]:
driver.quit()

In [17]:
'''
df.to_csv('Categories_&_links.csv')
'''

"\ndf.to_csv('Categories_&_links.csv')\n"

# Выбрали книги

In [18]:
df[(df['Subcategory'] == 'Books')]

,Category,Subcategory,Link
9,Books,Books,https://www.amazon.com/gp/browse.html?node=283...


In [19]:
df[(df['Subcategory'] == "Children's Books")]

,Category,Subcategory,Link
10,Books,Children's Books,https://www.amazon.com/gp/browse.html?node=4&r...


In [20]:
data = pd.DataFrame({'Title': [], 'Price': [], 'Currency':[], 'Rating': [], 'Number of reviews': [], 'Original Price': [], 'Main Genre': []})

In [29]:
def Start(c, category_name):
    logging.info("Start() function started")
    driver = webdriver.Chrome() # options=chrome_options
    flag = True
    while flag:
        try:
            if len(driver.find_elements(By.XPATH, "//div[@class = 'a-row a-text-center']//img")) != 0:
                driver = SolveCaptcha(driver)
            url = df[(df['Subcategory'] == c)]['Link'].iloc[0]
            
            driver.get(url)
            
            time.sleep(2)
            
            SetCountry(driver)
            driver.refresh()
            time.sleep(5)
            
            clickable = driver.find_element(By.ID, 'twotabsearchtextbox')
            
            time.sleep(1)
            
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            
            time.sleep(1) 
            
            ActionChains(driver)\
                        .send_keys(category_name)\
                        .perform()
                
            time.sleep(1)
            
            clickable = driver.find_element(By.ID, 'nav-search-submit-button')
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            flag = False
        except Exception as e:
            logging.error(f"Error appeared while starting driver for {category_name}", exc_info=e)
            break
    logging.info("Start() function ended")
    return driver

## Children's books

In [31]:
driver = Start("Children's Books", "Children's book")

In [32]:
logging.info(f"Scraping for {"Children's Books"} started")
for j in range(60):
    a = True
    flag = True
    while flag:
        try:
            if len(driver.find_elements(By.XPATH, "//div[@class = 'a-row a-text-center']//img")) != 0:
                driver = SolveCaptcha(driver)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            for i in soup.find_all('div', {'class': 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small'}):
                title = i.find('h2', {'class': 'a-size-base-plus a-spacing-none a-color-base a-text-normal'}).find('span').text
                try:
                    price = i.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text
                    currency = price[0]
                    price = float(price[1::])
                except:
                    price = np.nan
                    currency = np.nan
                    price = np.nan
                    
                try:
                    rating = float(i.find('a', {'class': 'a-popover-trigger a-declarative'}).get('aria-label').split()[0])
                    num_of_rev = int(i.find('div', {'class': 's-csa-instrumentation-wrapper alf-search-csa-instrumentation-wrapper'}).find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style'}).get('aria-label').split()[0].replace(',', ''))
                except:
                    rating = np.nan
                    num_of_rev = np.nan
                if price == 0:
                    try:
                        price = i.find('div', {'class': 'a-section a-spacing-none a-spacing-top-mini'}).find('div', {'class': 'a-row a-size-base a-color-secondary'}).find('span').split()[1]
                        currency = price[0]
                        price = float(price[1::])
                    except:
                        price = 0     
                
                try:
                    orig_price = float(i.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).text[1::])
                except:
                    orig_price = price
                data.loc[len(data)] = [title, price, currency, rating, num_of_rev, orig_price, "Children's Books"]
        
            clickable = driver.find_elements(By.CLASS_NAME, 's-list-item-margin-right-adjustment')[-1]
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            
            logging.info(f"Scraping page {j} for {"Children's Books"} ended")
            
            time.sleep(3)
            flag = True
        except Exception as e:
            a = False
            logging.error(f"Error appeared while scraping page {j} for {"Children's Books"}", exc_info=e)
            break
    if not a:
        break
            

In [26]:
data

,Title,Price,Currency,Rating,Number of reviews,Original Price,Main Genre
0,The Promise: A Cody and Summer tale,14.39,$,4.9,334.0,15.99,Children's Books
1,Cozy Critters on the Ark (Cozy Critters Storyt...,14.99,$,4.7,98.0,14.99,Children's Books
2,"Unicorns on Vacation: A story among dreams,fri...",13.99,$,NaN,NaN,13.99,Children's Books
3,The Rainbow Fish,5.04,$,4.8,6802.0,9.95,Children's Books
4,I Love You to the Moon and Back,4.92,$,4.9,77331.0,7.99,Children's Books
...,...,...,...,...,...,...,...
71,Where's Spot?,4.25,$,4.8,34840.0,7.99,Children's Books
72,You're My Little Cuddle Bug,5.08,$,4.9,8437.0,8.99,Children's Books
73,Funkelnde Träume und magische Wesen: Einhorngl...,14.60,$,NaN,NaN,14.60,Children's Books
74,Ode to a Pug,14.89,$,4.6,11.0,14.89,Children's Books


In [27]:
driver.quit()

In [28]:
def Scrape(n, driver, categ):
    logging.info(f"Scraping for {categ} started")
    for j in range(n):
        a = True
        flag = True
        while flag:
            try:
                if len(driver.find_elements(By.XPATH, "//div[@class = 'a-row a-text-center']//img")) != 0:
                    driver = SolveCaptcha(driver)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                for i in soup.find_all('div', {'class': 'a-section a-spacing-small a-spacing-top-small'}):
                    title = i.find('h2', {'class': 'a-size-medium a-spacing-none a-color-base a-text-normal'}).find('span').text
                    try:
                        price = i.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text
                        currency = price[0]
                        price = float(price[1::])
                    except:
                        price = np.nan
                        currency = np.nan
                        price = np.nan
                        
                    try:
                        rating = float(i.find('a', {'class': 'a-popover-trigger a-declarative'}).get('aria-label').split()[0])
                        num_of_rev = int(i.find('div', {'class': 's-csa-instrumentation-wrapper alf-search-csa-instrumentation-wrapper'}).find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style'}).get('aria-label').split()[0].replace(',', ''))
                    except:
                        rating = np.nan
                        num_of_rev = np.nan
                    if price == 0:
                        try:
                            price = i.find('div', {'class': 'a-section a-spacing-none a-spacing-top-mini'}).find('div', {'class': 'a-row a-size-base a-color-secondary'}).find('span').split()[1]
                            currency = price[0]
                            price = float(price[1::])
                        except:
                            price = 0     
                    
                    try:
                        orig_price = float(i.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).text[1::])
                    except:
                        orig_price = price
                    data.loc[len(data)] = [title, price, currency, rating, num_of_rev, orig_price, categ]
            
                clickable = driver.find_elements(By.CLASS_NAME, 's-list-item-margin-right-adjustment')[-1]
                ActionChains(driver) \
                    .click(clickable) \
                    .perform()
                
                time.sleep(3)
                flag = False
            except Exception as e:
                a = False
                logging.error(f"Error appeared while scraping page {j} for {categ}", exc_info=e)
                break
        if not a:
            break
        logging.info(f"Scraping page {j} for {categ} ended")
    return driver

In [29]:
fiction_categories = [
    "Comics & Manga",
    "Literature & Fiction",
    "Mystery, Thriller & Suspense",
    "Romance",
    "Science Fiction & Fantasy",
    "Teen & Young Adult"
]

nonfiction_categories = [
    "Arts & Photography",
    "Biographies & Memoirs",
    "Business & Money",
    "Calendars",
    "Computers & Technology",
    "Cookbooks, Food & Wine",
    "Crafts, Hobbies & Home",
    "Education & Teaching",
    "Engineering & Transportation",
    "Health, Fitness & Dieting",
    "History",
    "Humor & Entertainment",
    "Law",
    "Medical Books",
    "Parenting & Relationships",
    "Politics & Social Sciences",
    "Reference",
    "Religion & Spirituality",
    "Science & Math",
    "Self-help",
    "Sports & Outdoors",
    "Travel"
]

In [30]:
for categ in (fiction_categories + nonfiction_categories):
    driver = Start("Books", categ)
    driver = Scrape(50, driver, categ)
    driver.quit()
    data.to_csv('All_added_books_data_temp.csv')
    logging.info(f"Data for {categ} category added to All_added_books_data_temp.csv")


KeyboardInterrupt



In [ ]:
data

In [ ]:
data.to_csv('All_added_books_data.csv')
logging.info(f"Final data file compiled")

In [ ]:
data.drop_duplicates()